In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from typing import List

from utils import *
from solvers import EPOSolver, LinearScalarizationSolver
from models import *

import json
from pathlib import Path
import pygmo as pg

In [ ]:
data = pd.read_csv('data/powerp.csv', sep=';', decimal=',')
y = data[['PE']]
X = data.drop('PE', axis=1)

In [ ]:
torch.cuda.empty_cache()
scaler = StandardScaler()
seed = 1234
set_seed(seed)
set_logger()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Usando device = ', device)

hnet = POPI_hyper(input_target=4, target_hidden_dim=4, target_hidden_size=100).to(device)
net = POPI_target(input_target=4, target_hidden_dim=4, target_hidden_size=100).to(device)

# ---------
# Losses
# ---------

loss1 = AIW_loss()
loss2 = PICP_alpha_loss()


lr = 0.0005
wd = 0.
optimizer = torch.optim.Adam(hnet.parameters(), lr=lr, weight_decay=wd)

# ------
# solver
# ------

solvers = dict(ls=LinearScalarizationSolver, epo=EPOSolver)

solver_type = 'ls'
solver_method = solvers[solver_type]
if solver_type == 'epo':
    solver = solver_method(n_tasks=2, n_params=count_parameters(hnet))
else:
    # ls
    solver = solver_method(n_tasks=2)

# ------
# data
# ------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train)
y_val = scaler.transform(y_val)
y_test = scaler.transform(y_test)

train_features = torch.Tensor(X_train)
train_targets = torch.Tensor(y_train)
val_features = torch.Tensor(X_val)
val_targets = torch.Tensor(y_val)
test_features = torch.Tensor(X_test)
test_targets = torch.Tensor(y_test)
train_ds = TensorDataset(train_features, train_targets)
val_ds = TensorDataset(val_features, val_targets)
test_ds = TensorDataset(test_features, test_targets)

bs = 1500
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=len(val_ds), shuffle=True)
test_loader = DataLoader(test_ds, batch_size=len(test_ds), shuffle=True)

n_rays = 200
min_angle = 1e-3
max_angle = np.pi / 2 - 1e-3
test_rays = circle_points(n_rays, min_angle=min_angle, max_angle=max_angle)


In [ ]:
val_results = dict()
test_results = dict()
out_dir="outputs_PowerP"

no_val_eval = False
full_test = True
eval_every = 50
alpha=0.25

best_hv = -1
last_eval = -1
for epoch in range(0,12000):
    ###### Training ######
    
    for x_tr, y_tr in train_loader:
        hnet.train()
        optimizer.zero_grad()
        x_tr = x_tr.to(device)
        y_tr  = y_tr.to(device)
        
        if alpha > 0:
            ray = np.random.dirichlet((alpha, alpha), 1)
            ray = torch.from_numpy(ray.astype(np.float32).flatten()).to(device)
        else:
            alpha = torch.empty(1, ).uniform_(0., 1.)
            ray = torch.tensor([alpha.item(), 1 - alpha.item()]).to(device)
        
        weights = hnet(ray)
        low, up = net(x_tr, weights)
        
        p_int = torch.cat((low, up), axis=1)
        l1 = loss1(p_int, y_tr[:, [0]])
        l2 = loss2(p_int, y_tr[:, [0]])
        losses = torch.stack((l1, l2))
        
        ray = ray.squeeze(0)
        loss = solver(losses, ray, list(hnet.parameters()))
        
        loss.backward()
        optimizer.step()
    
    
    ###### HV Validation ######
    if (epoch + 1) >= 5000:
        if (epoch + 1) % eval_every == 0:
            last_eval = epoch
            if not no_val_eval:
                epoch_results, hv = evaluate(hypernet=hnet, targetnet=net, loader=val_loader, rays=test_rays, device=device)
                val_results[f'epoch_{epoch + 1}'] = epoch_results
                if hv > best_hv:
                    torch.save(hnet.state_dict(), 'best_model_hn_main.pt')
                    best_hv = hv
                    best_dict = epoch_results
                    best_epoch = (epoch + 1)
                print('Epoch: ', epoch+1, 'Best hv:', best_hv, 'in epoch', best_epoch)


In [ ]:
out_dir = Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

with open(Path(out_dir) / "val_results.json", "w") as file:
    json.dump(val_results, file)

In [ ]:
###### Test PIs #####

PINPS = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
hnet.load_state_dict(torch.load('best_model_hn_main.pt'))
for pinp in PINPS:
    test_epoch_results = test(hypernet=hnet, targetnet=net, loader=test_loader, val_dict=best_dict, PINP=pinp, rays=test_rays,device=device)
    test_results[f'PredInt_{pinp*100}'] = test_epoch_results
with open(Path(out_dir) / "test_results.json", "w") as file:
    json.dump(test_results, file)